In [2]:
%%capture
!pip install dart-fss  # DART API 라이브러리 설치

import dart_fss as dart
import pandas as pd

In [12]:
# 1. 환경 설정 및 데이터 로드
DART_API_KEY = '10ec77a9634c1f4a841fe87ddc921430b32f17f2'
dart.set_api_key(api_key=DART_API_KEY)
target_corp = dart.get_corp_list().find_by_corp_name('롯데칠성음료', exactly=True)[0]

# 2. 5개년 시계열 재무상태표 추출 (2020-2025)
fs = target_corp.extract_fs(bgn_de='20200101')
bs = fs['bs']

# 3. MultiIndex 컬럼 평면화 및 날짜 추출
new_columns = []
for col in bs.columns:
    date_label = [c for c in col if str(c).isdigit() and len(str(c)) == 8]
    new_columns.append(date_label[0] if date_label else col[-1])
bs.columns = new_columns

# 4. 데이터 필터링 및 단위 변환 (십억 원)
target_accounts = ['현금및현금성자산', '유형자산']
clean_df = bs[bs['label_ko'].isin(target_accounts)].copy()

numeric_cols = [c for c in clean_df.columns if str(c).isdigit()]
clean_df[numeric_cols] = clean_df[numeric_cols].apply(pd.to_numeric) / 1e9

# 5. 행/열 전환 및 정렬
final_df = clean_df.set_index('label_ko')[numeric_cols].T
final_df.index.name = '기준일자'
final_df.columns = ['현금(십억)', '유형자산(십억)']
final_df = final_df.sort_index()

# 6. 출력 옵션 및 시각화 (Colab 최적화)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.float_format', '{:.2f}'.format)

print(f"--- {target_corp.corp_name} 재무 데이터 정제 결과 ---")
try:
    from google.colab import data_table
    display(data_table.DataTable(final_df, include_index=True))
except:
    print(final_df)

# 7. CSV 저장 (Phase 2 입력 데이터)
final_df.to_csv('lotte_chilsung_financial_data.csv', encoding='utf-8-sig')

Annual reports:   0%|          | 0/6 [00:00<?, ?report/s]

--- 롯데칠성음료 재무 데이터 정제 결과 ---


,현금(십억),유형자산(십억)
기준일자,,
20171231,153.44,2245.31
20181231,98.39,2165.69
20191231,199.41,2014.59
20201231,158.85,1939.74
20211231,384.58,2048.22
20221231,345.00,2077.79
20231231,247.54,2439.02
20241231,177.84,2602.81
